# Required Python Packages

Let's install some python packages used in this notebook.

To import our own functions, you could upload the .py files in the input folder and do as we do for the "common.py" and "pytorch_common.py" files.

In [1]:
from shutil import copyfile

# public modules
import os
import sklearn
import torch
import torch.nn.functional as F
import random
import librosa
import os
import glob
import csv
import re
import itertools
import sys
import path
import numpy as np
import seaborn as sns
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [2]:
def time_mask(src_melspec, T, mT, p):
    melspec = np.copy(src_melspec)
    num_time_steps = melspec.shape[1] 
    for i in range(mT):
        t = 9999
        while(p*num_time_steps < t):
            t = np.uint16(np.random.uniform(0, T))
        t0 = np.uint16(np.random.uniform(1, num_time_steps - t))
        melspec[:,t0:t0+t+1] = melspec.mean() 
    return melspec
    
def frequency_mask(src_melspec, F, mF):
    melspec = np.copy(src_melspec)
    nmels = melspec.shape[0]
    for i in range(mF):
        f = np.uint16(np.random.uniform(0, F))
        f0 = np.uint16(np.random.uniform(0, nmels - f))
        melspec[f0:f0+f+1,:] = melspec.mean()
    return melspec

def data_aug(src_mel_spec, policy):
    new_mel_spec = frequency_mask(src_mel_spec,policy["F"], policy["mF"])
    new_mel_spec = time_mask(new_mel_spec,policy["T"] ,policy["mT"],policy["p"])
    return new_mel_spec

def augment_data_set(dataset, policy, N):
    aug_data = []
    names = []
    class_ = ''
    k = 0
    for i in tqdm(range(dataset.__len__())):
        item = dataset.__getitem__(i)
        aug_data.append(item[0])
        if(item[3] == 1): class_ = 'normal_' 
        else: class_ = 'anomaly_'
        names.append(class_ + 'id_' + str(item[2]) + str(k) + '_OG')
        k += 1
        for i in range(N):
            new_mel_spec = data_aug(item[0],policy)
            aug_data.append(new_mel_spec)
            names.append((class_ + 'id_' + str(item[2]) + str(k) + '_aug_'+ str(i)))
            k+=1
    return np.array(aug_data), np.array(names)

def augment_mel_spec(melspec, policy, N):
    new_melspecs = []
    new_melspecs.append(melspec)
    for i in range(N):
            new_mel_spec = data_aug(melspec,policy)
            new_melspecs.append(new_mel_spec)
    return np.array(new_melspecs)


In [3]:
def file_to_matrix_vector(folder_path, file_name,n_mels=64, N_crops = 5,n_fft=1024,hop_length=512,power=2.0, crop_width = 64):
    
    y, sr = librosa.load(os.path.join(folder_path,file_name), sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length,n_mels=n_mels,power=power)
    log_mel_spectrogram = 20.0 / power * np.log10(mel_spectrogram + sys.float_info.epsilon)
    
    dim_y = np.uint16(n_mels)
    dim_x = np.uint16(crop_width)    
    matrix_vector = np.zeros((N_crops, dim_y, dim_x))
    x = 0
    for i in range(N_crops):
        if(x+dim_x > log_mel_spectrogram.shape[1]):
                    x -= x+dim_x - log_mel_spectrogram.shape[1]
        matrix_vector[i, :, :] = log_mel_spectrogram[:,x:x+dim_x]
        x += dim_x
    return matrix_vector

def aug_log_mel_spec_to_matrix_vector(log_mel_spectrogram,n_mels=64,N_crops=5, crop_width = 64):
    '''
    convert a log_mel_spectrogram (array) to a vector(array)
    '''
    dim_y = np.uint16(n_mels)
    dim_x = np.uint16(crop_width)
    matrix_vector = np.zeros((N_crops, dim_y, dim_x))
    x = 0
    for i in range(N_crops):
        if(x+dim_x > log_mel_spectrogram.shape[1]):
                    x -= x+dim_x - log_mel_spectrogram.shape[1]
        matrix_vector[i, :, :] = log_mel_spectrogram[:,x:x+dim_x]
        x += dim_x
    return matrix_vector

In [4]:
def load_sg_to_array_Conv(folder, data_aug = {'augment' : False, 'policy' : None, 'N' : None} ,
                       n_mels=128, n_fft=1024, hop_length=512, power=2.0
                       ,N_crops=5, crop_width = 32):
    spectrograms = []
    i = 0
    filenames = os.listdir(folder)
    if data_aug['augment'] :
        for filename in tqdm(filenames):
            y, sr = librosa.load(os.path.join(folder,filename), sr=None)
            mel_spec = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length,n_mels=n_mels,power=power)
            log_mel_spectrogram = 20.0 / power * np.log10(mel_spec + sys.float_info.epsilon)
            
            aug_lms = augment_mel_spec(log_mel_spectrogram, data_aug['policy'], data_aug['N'])
            
            for aug_spec in (aug_lms):
                spec = aug_log_mel_spec_to_matrix_vector(aug_spec, n_mels, N_crops, crop_width)
                spectrograms[i:i+ len(spec)] = spec[:]
                i += len(spec)    
    else :
        for filename in tqdm(filenames):
            spec = file_to_matrix_vector(folder,filename,n_mels,N_crops,n_fft,hop_length,power, crop_width)
            spectrograms[i:i + len(spec)] = spec[:]
            i += len(spec)
    return np.array(spectrograms)

**BASELINE VAR**

In [5]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import keras
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from keras.models import Model, load_model
from keras.layers import Lambda, Input, Dense, Flatten, BatchNormalization, Activation, Reshape
from keras.layers import Conv2D, Cropping2D, Conv2DTranspose
from keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers
from keras import backend as K
from keras import initializers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
A = {"W" : 20, "F" : 27, "mF" : 1, "T" : 100, "p" : 1.0, "mT" : 1}
B = {"W" : 20, "F" : 27, "mF" : 2, "T" : 100, "p" : 1.0, "mT" : 2}
C = {"W" : 10, "F" : 15, "mF" : 2, "T" : 70, "p" : 0.2, "mT" : 2}
D = {"W" : 10, "F" : 27, "mF" : 2, "T" : 70, "p" : 1.0, "mT" : 2}
E = {"W" : 10, "F" : 15, "mF" : 1, "T" : 70, "p" : 1.0, "mT" : 1}

params_conv = {'latent_dim' : 20}

data_augmentation = {'augment' : True, 'policy' : E, 'N' : 1}
ms_prms = {"n_mels" : 256,"n_fft" : 1023,"hop_length" : 512,"power" : 2.0}

N_crops = 9
crop_width = 32

train_dir = '../input/eurecom-aml-2022-challenge-2/eval_data/eval_data/slider/train'
test_dir = '../input/eurecom-aml-2022-challenge-2/dev_data/dev_data/slider/test'

In [7]:
X_train  = load_sg_to_array_Conv(train_dir, data_aug = {'augment' : False, 'policy' : E, 'N' : 1},
                                 n_mels=ms_prms['n_mels'], n_fft=ms_prms['n_fft'], hop_length=ms_prms['hop_length'], power=ms_prms['power'],
                                 N_crops=N_crops, crop_width=crop_width)
# X_aug_train = load_sg_to_array_Conv(train_dir, data_aug = data_augmentation,
#                                 n_mels=ms_prms['n_mels'], n_fft=ms_prms['n_fft'], hop_length=ms_prms['hop_length'], power=ms_prms['power'], 
#                                 N_crops=N_crops, crop_width=crop_width)
X_test  = load_sg_to_array_Conv(test_dir, data_aug = {'augment' : False, 'policy' : E, 'N' : 1},
                                 n_mels=ms_prms['n_mels'], n_fft=ms_prms['n_fft'], hop_length=ms_prms['hop_length'], power=ms_prms['power'], 
                                 N_crops=N_crops, crop_width=crop_width)


x_tst = X_test[:,:,:,None]
x_tr = X_train[:,:,:,None]
#x_aug_tr = np.copy(X_aug_train)[:,:,:,None]

100%|██████████| 1101/1101 [00:49<00:00, 22.12it/s]


In [8]:
test_file_list = os.listdir(test_dir)
y_test_crops = np.empty(x_tst.shape[0])
y_test = np.empty(len(test_file_list))

c = 0
for idx, file in tqdm(enumerate(test_file_list)):
    if file.find("normal") == -1:
        y_test[idx] = True
        y_test_crops[c:c+N_crops] = True
    else: 
        y_test[idx] = False
        y_test_crops[c:c+N_crops] = False
    c+=N_crops

1101it [00:00, 396763.36it/s]


In [9]:
def combine_predicts(crop_predictions, mode='mean'):
    c = 0
    p = 0
    predictions = []
    while c <= len(crop_predictions)-5:
        pred = crop_predictions[c:c+5]
        if mode == 'min':
            predictions.append(np.min(pred))
        elif mode == 'max':
            predictions.append(np.max(pred))
        elif mode == 'mean':
            predictions.append(np.mean(pred))
        else:
            raise ValueError('Unknown mode')
        p+=1
        c+=N_crops
    return np.array(predictions)

In [10]:
original_dim = X_train.shape[1:3]
input_shape = (original_dim,)
latent_dim = params_conv['latent_dim']

In [11]:
input_img = Input(shape=(original_dim[0], original_dim[1], 1))
initializer = tf.keras.initializers.GlorotNormal()
#initializer = tf.keras.initializers.Zeros()
x = Conv2D(32, (5, 5),strides=(2,1), padding='same',kernel_initializer=initializer)(input_img)   #256x64 -> 128x64
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(64, (5, 5),strides=(2,2), padding='same',kernel_initializer=initializer)(x)           #64x64 Changed stride
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(128, (5, 5),strides=(2,1), padding='same',kernel_initializer=initializer)(x)          #32x32
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3),strides=(2,2), padding='same',kernel_initializer=initializer)(x)          #16x16
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3),strides=(2,2), padding='same',kernel_initializer=initializer)(x)          #8x8
x = BatchNormalization()(x)
x = Activation('relu')(x)

#temp_model = Model(inputs=input_img, outputs=x)
#temp_model.summary()

2022-05-25 18:05:20.072159: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 18:05:20.073904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 18:05:20.074951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [12]:
volumeSize = K.int_shape(x)

x = Conv2D(latent_dim, (4,4), strides=(1,1), padding='valid')(x)
encoded = Flatten()(x)
# z_mean = Dense(latent_dim)(encoded)
# z_log_var = Dense(latent_dim)(encoded)
# z = Lambda(sample, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

In [13]:
x = Dense(volumeSize[1] * volumeSize[2] * volumeSize[3])(encoded) 
x = Reshape((volumeSize[1], volumeSize[2], 512))(x)                #8x8

x = Conv2DTranspose(256, (3, 3),strides=(2,2), padding='same',kernel_initializer=initializer)(x)  #16x16
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2DTranspose(128, (3, 3),strides=(2,2), padding='same',kernel_initializer=initializer)(x)  #32x32   
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2DTranspose(64, (5, 5),strides=(2,2), padding='same',kernel_initializer=initializer)(x)   #64x64
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2DTranspose(32, (5, 5),strides=(2,1), padding='same',kernel_initializer=initializer)(x)   #128x64
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2DTranspose(16, (5, 5),strides=(2,1), padding='same',kernel_initializer=initializer)(x)   #256x64
x = BatchNormalization()(x)
x = Activation('relu')(x)

decoded = Conv2DTranspose(1, (5, 5), padding='same')(x)
conv_AE = Model(inputs=input_img, outputs=decoded)
# conv_AE.summary()

In [14]:
opt = optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
#opt = optimizers.RMSprop(learning_rate=0.0001)
top_score = 0
weight_best_model = None
conv_AE.compile(optimizer=opt, loss="mse")
conv_AE.summary()
# Finally, we train the model:

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 32, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 32, 32)       832       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 32, 32)       128       
_________________________________________________________________
activation (Activation)      (None, 128, 32, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 16, 64)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 16, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 16, 64)        0     

In [15]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        X_test_pred = conv_AE.predict(x_tst)[:,:,:,0]
        x_tst_3 = x_tst[:,:,:,0]
        mae_vector_test = np.zeros(X_test_pred.shape[0])
        for idx, x_pred in tqdm(enumerate(X_test_pred)):
            mae_vector_test[idx] = np.mean((np.absolute((x_pred-x_tst_3[idx]))))
        y_test_predict = combine_predicts(mae_vector_test, mode='mean')
        auc_val_score = roc_auc_score(y_test, y_test_predict)
        print(auc_val_score)
        if auc_val_score > top_score:
            weight_best_model = conv_AE.get_weights()

In [16]:
results = conv_AE.fit(x_tr, x_tr,shuffle=True,epochs=20, batch_size=254, callbacks=[CustomCallback()])

Train on 21330 samples
Epoch 1/20


2022-05-25 18:05:28.740134: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-25 18:05:28.741400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 18:05:28.743051: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 18:05:28.744613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

21330/21330 [==============================] - ETA: 0s - loss: 627.1861

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
9909it [00:00, 15784.53it/s]


0.2131960049937578
21330/21330 [==============================] - 45s 2ms/sample - loss: 627.1861
Epoch 2/20
21330/21330 [==============================] - ETA: 0s - loss: 74.7307

...

9909it [00:00, 10408.21it/s]

0.7542238868081566
21330/21330 [==============================] - 27s 1ms/sample - loss: 16.0088


Epoch 20/20
21330/21330 [==============================] - ETA: 0s - loss: 15.9268

9909it [00:00, 16851.51it/s]


0.7573616312942156
21330/21330 [==============================] - 27s 1ms/sample - loss: 15.9268


In [17]:
#conv_AE.set_weights(weight_best_model) 
X_test_pred = conv_AE.predict(x_tst)[:,:,:,0]
x_tst_3 = x_tst[:,:,:,0]
mae_vector_test = np.zeros(X_test_pred.shape[0])
for idx, x_pred in tqdm(enumerate(X_test_pred)):
    mae_vector_test[idx] = np.mean((np.absolute((x_pred-x_tst_3[idx]))))

9909it [00:00, 17118.26it/s]


In [18]:
y_test_predict = combine_predicts(mae_vector_test, mode='mean')

In [19]:
roc_auc_score(y_test, y_test_predict)

0.7573616312942156

## Submission

In [20]:
import gc
gc.collect()
del X_train, x_tr
submit_dir = '../input/eurecom-aml-2022-challenge-2/eval_data/eval_data/slider/test'
X_submit  = load_sg_to_array_Conv(submit_dir, data_aug = {'augment' : False, 'policy' : E, 'N' : 1},
                             n_mels=ms_prms['n_mels'], n_fft=ms_prms['n_fft'], hop_length=ms_prms['hop_length'], power=ms_prms['power'], 
                             N_crops=N_crops, crop_width=crop_width)
x_sbmt = np.copy(X_submit)[:,:,:,None]
x_sbmt_3 = x_sbmt[:,:,:,0]
X_submit_pred = conv_AE.predict(x_sbmt)[:,:,:,0]
mae_vector_submit = np.zeros(X_submit_pred.shape[0])
for idx, x_pred in tqdm(enumerate(X_submit_pred)):
    mae_vector_submit[idx] = np.mean((np.absolute((x_pred-x_sbmt_3[idx]))))
y_submit_predict = combine_predicts(mae_vector_submit, mode='max')

100%|██████████| 834/834 [00:37<00:00, 22.46it/s]
7506it [00:00, 16320.33it/s]


In [21]:
import pandas as pd
submit_files = os.listdir(submit_dir)
df_submit = pd.DataFrame({'file_name': submit_files, 'anomaly_score': y_submit_predict})
df_submit.to_csv('submission.csv', index=False)